# How to infer a stellar velocity.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import pymc3 as pm

import aviary as av

First, you need to construct a prior. We're going to use the velocities of stars in the Kepler field to make a prior because our target stars will also be in the Kepler field. If you're not looking that the Kepler field it's important that you build a custom prior. You'll need to calculate the mean and covariance over the three velocities and ln(distance): Vx; Vy; Vz; lnD for stars in your field of interest. 

In [2]:
mu, cov = av.get_prior()

/Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Now let's choose a target star.

In [3]:
kepler_data = av.kepler_data()
df = kepler_data.iloc[0]  # Select the first star in our Kepler data catalog.

In [4]:
# Set up the position and proper motion variables.
pos = [float(df["ra"]), float(df["dec"]), float(df["parallax"])]
pos_err = [float(df["ra_error"]), float(df["dec_error"]),
           float(df["parallax_error"])]
proper = [float(df["pmra"]), float(df["pmdec"])]
proper_err = [float(df["pmra_error"]), float(df["pmdec_error"])]

Run PyMC3

In [ ]:
trace = av.run_pymc3_model(pos, pos_err, proper, proper_err, mu, cov)

optimizing logp for variables: [vxyzD]


message: Optimization terminated successfully.
logp: -9391.024645875093 -> -8.185998468490368
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vxyzD]
Sampling 4 chains, 0 divergences:  94%|█████████▎| 9367/10000 [00:05<00:00, 2771.21draws/s]

In [ ]:
flat_samples = pm.trace_to_dataframe(trace)
params_inferred = np.median(flat_samples, axis=0)
upper = np.percentile(flat_samples, 84, axis=0)
lower = np.percentile(flat_samples, 16, axis=0)
errp = upper - params_inferred
errm = params_inferred - lower
std = np.std(flat_samples, axis=0)

results = pd.DataFrame(dict({
    "kepid": df.kepid,
    "vx_inferred": params_inferred[0],
    "vx_inferred_errp": errp[0],
    "vx_inferred_errm": errm[0],
    "vx_inferred_err": std[0],
    "vy_inferred": params_inferred[1],
    "vy_inferred_errp": errp[1],
    "vy_inferred_errm": errm[1],
    "vy_inferred_err": std[1],
    "vz_inferred": params_inferred[2],
    "vz_inferred_errp": errp[2],
    "vz_inferred_errm": errm[2],
    "vz_inferred_err": std[2],
    "lndistance_inferred": params_inferred[3],
    "lndistance_inferred_errp": errp[3],
    "lndistance_inferred_errm": errm[3],
    "lndistance_inferred_err": std[3]
    }), index=[0])

In [ ]:
results